Membres de l'équipe: Josue ABOU MAWUFEMO , Kawtar MOULAHID, Abderrahmane BAROUALI

<h1>Sentiment analysis</h1>
<ol>
<h3><li>Définition du Sujet et Objectif</h3></li>
<b>Sentiment analysis ?</b><br>
<p>Aussi connue sous le nom de <strong><i>« Opinion Mining »</i></strong>, <strong>l’analyse des sentiments</strong> consiste à identifier les informations subjectives d’un texte pour <strong>extraire l’opinion de l’auteur</strong>. <br>
De manière générale, <strong>l’analyse des sentiments</strong> permet de mesurer le niveau de satisfaction des clients vis-à-vis des produits ou services fournis par une entreprise ou un organisme. Elle peut même s’avérer <strong>bien plus efficace que des méthodes classiques</strong> comme les sondages puisque <strong>de nos jours, une partie croissante des consommateurs partage fréquemment leurs opinions sur les réseaux sociaux</strong>. </p>

<p><i><strong>Objectif du Projet:</strong></i> Notre but est donc de mettre en place un modèle permettant de classifier les différents avis des clients d’un hôtel en deux classes qui sont : Avis positifs et avis négatifs.<br>
Cette classification permettra au personnel de l’hôtel de pouvoir identifier les principales plaintes (Texte négatifs) afin d’améliorer leurs services et réduire le niveau d’insatisfaction des clients.<br></p>

<h3><li>Constitution de la dataset</h3></li>
<p>Avant d’entamer le travail proprement dit, nous allons d’abord décrire brièvement notre dataset.<br>
Nous avons utilisé une dataset téléchargeable sur Kaggle via le lien suivant : 
<a href="https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews">https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews</a><br></p>


<p>Aperçu de la dataset</p>

In [2]:
import pandas as pd
df = pd.read_csv('all_reviews_final.csv', sep=';', on_bad_lines='skip')

df.to_csv('all_reviews_final.csv', index=False)  

df.isna().sum()
df.head()

,Review,Rating
0,Hollywood Hotel sure lives up to its name. Yo...,5
1,We had a reservation at this hotel this weeken...,5
2,We were mostly happy with our stay here. The s...,4
3,Pil give us great customer service. He took go...,5
4,Very friendly staff. Daniel did an excellent j...,5


En affichant le résumé de la dataset, on voit qu’elle contient environ 20 000 lignes et deux colonnes : <br>
<ul>
    <li>Une colonne Review : qui n’est autre qu’un paragraphe contenant le commentaire de l’utilisateur</li>
    <li>Une colonne Rating : qui précise s’il s’agit d’un avis positif ou pas. Notre objectif principal est de <b>détecter les avis négatifs</b> donc la <b>classe positive(1)</b>correspond aux avis négatifs et la <b>classe négative(0)</b> correspond aux avis positifs.</li>
</ul>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35578 entries, 0 to 35577
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  35578 non-null  object
 1   Rating  35578 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 556.0+ KB


<p>En affichant les quantités de données pour chaque catégorie, on s’aperçoit que la dataset est disproportionnée. Elle contient plus d’avis positifs (environ 17000) que d’avis négatifs (environ 3000). Cette différence pourra sans doute influencer le modèle lors de l’entrainement. Ce dernier risque donc d’overfitter sur les avis positifs.</p>

In [4]:
df.Rating.value_counts()

5    15104
4     9581
3     5243
1     2942
2     2708
Name: Rating, dtype: int64

Pour remédier à cela, il existe plusieurs possibilités :
<ul>
    <li><b>Sous-échantillonnage de la classe majoritaire</b> qui consiste à réduire le nombre d'échantillons de la classe majoritaire en sélectionnant au hasard un sous-ensemble de données de cette classe à utiliser pour l'entrainement.</li>
    <br>
    <li><b>Suréchantillonnage de la classe minoritaire </b>qui consiste à augmenter le nombre de données de la classe minoritaire dans l'ensemble de données d'apprentissage.</li>
     <br>
    <li>Faire du <b>webscraping</b> afin d’augmenter le nombre d’avis négatifs.</li>

</ul>


L'un des principaux inconvénients du <b>sous-échantillonnage</b> est que des données ou des informations utiles peuvent être perdues. De plus dans notre cas, on obtiendra en tout environ 6000 lignes de données, ce qui n’est pas vraiment approprié pour un problème de NLP.<br><br>
Le principal inconvénient du <b>suréchantillonnage </b> est qu'elle peut entraîner un overfitting. <br><br>
Nous allons donc opter pour la troisième solution qui est celle du <b>webscraping</b>.<br><br>
Nous avons scrapper le site <a href="https://www.tripadvisor.com">www.tripadvisor.com</a>. Les détails du webscraping se trouve dans le notebook <a href="webscraping_NoteBook.ipynb"> webscraping_NoteBook.ipynb</a>. <br><br>
Grâce au webscraping on a pu obtenir environ 5500 lignes avis négatifs. La dataset contient désormais 25000 données. Cette fois ci, en faisant un sous-échantillonnage, on obtient 17000 données en tout (8500 pour chaque catégorie)<br>


In [12]:
import pandas as pd


df = pd.read_csv('all_reviews_final.csv', sep=',', on_bad_lines='skip')
df = df.sample(n=8500, replace=True)



#print(df['Rating'].value_counts())
df

,Review,Rating
28296,I have stayed at The Jane 3 times. Each time w...,5
16713,This hotel should not be a four star hotel. AC...,1
31462,"The location is great, and for that price we d...",2
4317,Rooms are decorated with actual colors and see...,4
13482,I stayed at the hotel for one week. The room ...,5
...,...,...
30330,I stayed at the New Yorker during a weekend of...,4
34982,The employees at the Ameritania went out of th...,5
7944,I loved that we were able to walk to city walk...,3
34212,We were surprised at the size of our room - it...,4


Une fois que cela est fait, Notre dataset est constitué et on peut enchainer avec les étapes suivantes.


<ol start="3">
<h3><li>Plan de travail</h3></li>
Comme on peut s’en douter, Il s’agit ici d’un problème de Natural Language Processing (NLP) ou Traitement automatique du langage naturel.<br>
Et comme pour tout problème de NLP, Notre travail se compose principalement de deux parties :

<ul>
    <li>La partie <b>« linguistique »</b>, qui consiste à prétraiter et transformer les informations en entrée en un jeu de données exploitable.</li>
    <li>La partie <b>« apprentissage automatique »</b>, qui porte sur l’application de modèles <b><i>de Machine Learning</i></b> à ce jeu de données.</li><br>
</ul>
Dans la suite du rapport, Nous allons aborder ces deux aspects, en décrivant brièvement les <b>principales méthodes utilisées </b>et en précisant les principaux défis auxquels nous avons fait face.
</ol>
<ol type="A">
<h3><li>Partie linguistique : Du texte à la donnée</li></h3>
<p>Avant toute chose, il est important de s'assurer que notre dataset ne contient pas de données manquantes</p>
<p>Parmi les principales étapes de cette partie, on retrouve :</p>
<ol type="a">
<li>Nettoyage : cette phase consiste à réaliser des tâches telles que la <b><i>suppression d’urls, d’emoji, suppression de  ponctuation, de symboles et de stopwords, passage en minuscule</i><b>.</li>
</ol>
</ol>


Ci-dessous les fonctions permettant de faire le nettoyage.

Pour que le code ci-dessous s'exectue correctement, if faut tout d'abord installer la bibliothèque nltk avec la commande <i> piip install nltk </i>. <br> Ensuite, il faute lancer le CLI de python et excetuer les commandes suivantes afin de télécharger le package <i>stopwords</i><br>
>>>import nltk <br>
>>>nltk.download('stopwords')

In [13]:
import nltk 
import re
import string
from nltk.corpus import stopwords

def remove_URL (text):
    url = re.compile(r"https?://\s+ www\.\s+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
         

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (i0s)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)


def remove_punct (text):
    table = str.maketrans ("","",string.punctuation)
    return text.translate(table)

stop = set (stopwords.words ( "english"))
def remove_stopwords (text):
    text = [word.lower () for word in text.split() if word.lower() not in stop]
    return " ".join (text)

<p>On regroupe toutes ses fonctions en une seule appelée <i>nettoyerDataframe</i></p>

In [14]:
def nettoyerDataframe(df):
    Review_cleaned = df.Review.map(lambda x: remove_URL(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_html(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_emoji(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_punct(x)) 
    Review_cleaned = Review_cleaned.map(remove_stopwords)   
    df.insert(1, "Review_cleaned", Review_cleaned)
    return df

Ensuite on applique cette fonction pour nettoyer la colonne « Review » et on stocke le résultat dans la colonne <i>« Review_cleaned »</i>


In [15]:
df = nettoyerDataframe(df)

df.head()

,Review,Review_cleaned,Rating
28296,I have stayed at The Jane 3 times. Each time w...,stayed jane 3 times time fun first lobby histo...,5
16713,This hotel should not be a four star hotel. AC...,hotel four star hotel ac loud bathroom standar...,1
31462,"The location is great, and for that price we d...",location great price didnt exactly expect gold...,2
4317,Rooms are decorated with actual colors and see...,rooms decorated actual colors seems clean wifi...,4
13482,I stayed at the hotel for one week. The room ...,stayed hotel one week room clean good smell st...,5


<ol type="a" start="2">
<li>Normalisation des données :</li>
<br>
<b><i>Tokenisation</i></b>, ou découpage du texte en plusieurs pièces appelés tokens.<br><br>
Pour cela, nous utiliserons la méthode des N-grammes.
<br><br>
Les N-grammes sont simplement toutes les combinaisons de mots ou de lettres adjacents de longueur n que nous pouvons trouver dans notre texte source. Les ngrammes avec n=1 sont appelés <i>unigrammes</i>. De même, <i>les bigrammes</i>(n=2), les trigrammes (n=3) et ainsi de suite peuvent également être utilisés.
<br><br>
Les unigrammes ne contiennent généralement pas beaucoup d'informations par rapport aux bigrammes et aux trigrammes. Le principe de base derrière les n-grammes est qu'ils capturent la lettre ou le mot susceptible de suivre le mot donné. Plus le n-gramme est long (n élevé), plus vous devez travailler avec du contexte.
<br><br>
Dans le cas de notre projet, Nous utiliserons les <b><i>unigrammes</i></b> puisque :
<ul>
<li>L’utilisation des N-grammes avec N>=2 conduit à une Memory Error (Mémoire RAM insuffisante).</li>
</ul>
<br><br>
<li> Ensuite, Afin de pouvoir appliquer les méthodes de <i>Machine Learning</i> aux problèmes relatifs au langage naturel, il est indispensable de <i>transformer les données textuelles en données numériques</i>.<br><br>Pour se faire, Il existe plusieurs approches. L’une d’entre elles que nous utiliserons dans ce projet est celle du TF-IDF (<i>Term Frequency-Inverse Document Frequency</i>). Cette méthode consiste <b>à compter le nombre d’occurrences des tokens</b> présents dans le corpus pour chaque texte, que l’on divise ensuite par le nombre d’occurrences total de ces même tokens dans tout le corpus.</li>
</ol>

Le TfidfVectorizer de la bibiliothèque sklearn.features_extraction.text de python, permet à la fois de faire la tokénization et la conversion en tfidf.<br><br>On applique le TfidfVectorizer à la colonne Review_cleaned et on stocke le résultat dans la variable X qui servira plutard pour l'entrainement des modeles.
<br><br>On affiche les valeurs non-nulles du vecteur <i>tfidf</i> obtenu pour la première ligne juste pour voir à quoi ressemblent les coefficients tfidf.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))

X = tfidf_vectorizer.fit_transform(df.Review_cleaned)

[ x for x in X.todense()[0][0:].tolist()[0] if x != 0] 

[0.29535419680594627,
 0.05832342764110454,
 0.17427056192765256,
 0.16978095824054404,
 0.12129956369605584,
 0.08767363184296899,
 0.11502903771834433,
 0.06015903068253284,
 0.18821654476794925,
 0.07692565515502141,
 0.15583497540024738,
 0.11611685941221411,
 0.14906522119831156,
 0.13121285812352182,
 0.08848709718260646,
 0.15795480793310407,
 0.18821654476794925,
 0.09955423360525187,
 0.20490455393434637,
 0.12557323856540223,
 0.1282125316623767,
 0.10361896901011615,
 0.13441320801170953,
 0.12845239895992713,
 0.10189335768536174,
 0.028151452562497743,
 0.09373548068808747,
 0.118486630566466,
 0.17427056192765256,
 0.0975698055621141,
 0.10259446696027558,
 0.12117325551771826,
 0.06102586383111474,
 0.06607451410388257,
 0.07434889287591084,
 0.09036025679422123,
 0.1328048685786113,
 0.18005866777067497,
 0.18821654476794925,
 0.06045555673015557,
 0.06167716302090115,
 0.045249546981836836,
 0.12266429311690476,
 0.054232881071335344,
 0.03631462215490551,
 0.193663108

<ol type="A" start="2">
<h3><li>La phase d'apprentissage: Des données au modèle</li></h3>
<p>De manière globale, on peut distinguer <b>3 principales approches NLP</b> : les <b>méthodes basées sur des règles</b>, modèles classiques de Machine Learning et modèles de Deep Learning.</p>
<br>
<ol type="a">
<li>Modèles utilisés</li>
<p>Nous utiliserons uniquement les modèles classiques de Machine Learning.<br>Elles mettent généralement en œuvre un modèle <b>statistique d’apprentissage automatique</b> tels que ceux de <b>Naive Bayes</b>, de <b>Régression Logistique</b>, <b>SVM</b>.
<br><br>
Nous utiliserons plusieurs modèles  et nous ferons ensuite du cross_validation afin des les comparer .</p>
</ol>
</ol>

<img src="image.jpg" style="width:800px;height:500px; display: block; margin-left: auto;margin-right: auto;">

Avant de passer à l'entraînement des modèles, il reste encore une chose importante à faire qui est la définition des métriques d'évaluation 

<ol type="a" start="2">
<li>Métriques d’évaluation </li>
<br>
Après, Donc, afin d'évaluer les modèles de classification, nous utiliserons ces métriques :
<ul>
<li><b>Accuracy:</b> quantifie le nombre de prédictions correctes</li>
<li><b>Précision:</b> quantifie le nombre de détections positives (avis négatifs) appartenant réellement à la classe positive.</li>
<li><b>Rappel:</b> quantifie le nombre détections positives à partir de l’ensemble des exemples positifs du jeu de test.</li>
<li><b>Score F1:combine subtilement la précision et le rappel. Il est intéressant et plus intéressant que l’accuracy car le nombre de vrais négatifs (tn) n’est pas pris en compte.</b></li>
</ul>
</ol>

L’objectif de notre projet étant de détecter les avis négatifs, nous allons privilégier le rappel par rapport à la précision. Puisqu’en soit, il est préférable de classifier un avis positif comme étant négatif plutôt que de plutôt que l’inverse, c’est-à-dire classifier un avis négatif comme positif.<br><br>
Toutefois, nous évaluerons quand même l’ensemble de ces métriques.<br><br>
Maintenant nous allons entrainer les modèles et faire les prédictions.

<h4>Définition de la fonction permettant de faire la validation croisée et l'affichage des métriques d'évaluation</h4>

In [28]:
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.model_selection import cross_val_score

def cross_validation(model, X, y):
    rappels = cross_val_score(model, X,y,scoring='recall')
    rappels = pd.Series(rappels)
    print("Rappels : min=",rappels.min(),"  max=",rappels.max(),"  moyenne=",rappels.mean())

    precisions = cross_val_score(model, X,y,scoring='precision')
    precisions = pd.Series(precisions)
    print("Precisions : min=",precisions.min(),"  max=",precisions.max(),"  moyenne=",precisions.mean())

    accuracys = cross_val_score(model, X,y,scoring='accuracy')
    accuracys = pd.Series(accuracys)
    print("Accuracy : min=",accuracys.min(),"  max=",accuracys.max(),"  moyenne=",accuracys.mean())

    f1_scores = cross_val_score(model, X,y,scoring='f1')
    f1_scores = pd.Series(f1_scores)
    print("F1_score : min=",f1_scores.min(),"  max=",f1_scores.max(),"  moyenne=",f1_scores.mean())


<h3>Entraînement des modèles et affichage des métriques d'évaluation après validation croisée</h3>

On définit la variable "y" qui correspond aux labels

In [29]:
y = df["Rating"].values

<h4>Regression logistique</h4>

In [30]:
#Regression logistique
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression()
logisticRegression.fit(X, y)


print("\n***** Métriques d'évaluation : Regression logistique ******")
cross_validation(logisticRegression,X,y)

C:\Users\bouta\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



***** Métriques d'évaluation : Regression logistique ******


C:\Users\bouta\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 1

Rappels : min= nan   max= nan   moyenne= nan


C:\Users\bouta\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 1

Precisions : min= nan   max= nan   moyenne= nan


KeyboardInterrupt: 

<h4>Naives Bayes</h4>

In [26]:
#Naives Bayes
from sklearn.naive_bayes import GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X.toarray(), y)


print("\n***** Métriques d'évaluation : Naives Bayes ******")
cross_validation(gaussianNB,X,y)


***** Métriques d'évaluation : Naives Bayes ******
Rappels : min= nan   max= nan   moyenne= nan
Precisions : min= nan   max= nan   moyenne= nan
Accuracy : min= nan   max= nan   moyenne= nan
F1_score : min= nan   max= nan   moyenne= nan


C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 245, in fit
    return self._partial_fit(
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 402, in _partial_fit
    X, y = self._validate_data(X, y, reset=first_call)
  File "C:\Users\bouta\anacond

<h4>Support Vector Machine</h4>

In [27]:
#SVM
from sklearn import svm

svmModel = svm.SVC(kernel='linear') # Linear Kernel
svmModel.fit(X, y)


print("\n***** Métriques d'évaluation : SVM ******")
cross_validation(svmModel,X,y)


***** Métriques d'évaluation : SVM ******


C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\bouta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 

KeyboardInterrupt: 

<h3>Conclusion</h3>
Comme explicité un peu plus haut, la métrique qui nous intéresse est le rappel, et pour les différents modèles que nous avons testé celui qui nous permet d’obtenir un meilleur rappel est la régression logistique. <br><br>
On a : …. pour la regression logistique, …… pour le Naive Bayes et ……. Pour le SVM. <br><br>
Toutefois il est important de noter que le fait d’obtenir un rappel de ….. ne garantit pas que lorsque le modèle sera en production on aura toujours ce même rappel. Mais on sait que cela est probable.